In [ ]:
# Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Reading the csv file
data = pd.read_csv('/kaggle/input/pakistans-largest-ecommerce-dataset/Pakistan Largest Ecommerce Dataset.csv')
df = data.copy()

FileNotFoundError: ignored

## Step 1: Data Pre-processing

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
# Checking for missing / NaN values
df.isnull().sum()

In [ ]:
# Doing a visual inspection of all columns
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')

##### Observations
- Out of 26 columns, last 5 columns in the dataset contain NaN values for all records
- Records at 464051 indices (from the bottom) contain NaN values for all columns
- ' MV ' is an ambiguous column name with extra spaces
- Some of the columns have incorrect data types

##### Actions
- Last 5 columns need to be dropped from the dataset
- 464051 rows, containing NaN values need to be dropped from the dataset
- Renamed the columns ' MV ' and 'category_name_1' to 'MV' and 'category_name'

In [ ]:
df.drop(["Unnamed: 21", "Unnamed: 22", "Unnamed: 23", "Unnamed: 24", "Unnamed: 25"], axis = 1, inplace=True)
df.dropna(subset=["item_id"], axis=0, inplace=True)
df.rename(columns={" MV ": "MV", "category_name_1": "category_name"}, inplace = True)

##### Dropping duplicate entries, if any, from the dataset

In [ ]:
df=df.drop_duplicates()

##### Basic data quality and integrity checks

In [6]:
print("The number of rows with negative or zero Quantity:",sum(n <= 0 for n in df.qty_ordered))
print("The number of rows with negative Price:",sum(n < 0 for n in df.price))

NameError: ignored

##### Convert all values in 'sku' column to upper case for uniformity

In [ ]:
df['sku']=df['sku'].str.upper()

#### Exploring all columns, finding and Imputing Null Values
#### Categorical Variables

In [ ]:
df['status'].value_counts()

##### Observations
- There are a lot of labels for 'status' column.
- Need to check if any relationship exists between 'status' and 'BI Status' columns

In [ ]:
df.groupby('BI Status')['status'].value_counts()

##### Observations
- All transactions marked as either **'complete' or 'closed'**, fall in the **'Net' category** for 'BI Status'
- All transactions marked as **'received','paid','cod','exchanged' or something related to refund** are marked in **'Valid' category**
- All transactions marked as **either 'canceled' or something to do with incomplete transation** are marked in **'Gross' category**
- '#REF!' looks an erroneus label.

##### Actions
**Replace values inside the 'status' column by creating new labels**

- **'complete','closed','received','paid','cod'** will belong to category **'Completed'**
- **'order_refunded','refund', 'exchange'** will belong to category **'Refund'**
- **'pending','payment_review','processing','holded','pending_paypal','\N'** will beling to **'Pending'**
- **'canceled'** will belong to **'Cancelled'**
- **'fraud'** will belong to **'Fraud'**
**Also replace the '#REF!'' entry to 'Net' in 'BI status'**

In [ ]:
df['status'] = df['status'].replace('complete', 'Completed')
df['status'] = df['status'].replace('closed', 'Completed')
df['status'] = df['status'].replace('received', 'Completed')
df['status'] = df['status'].replace('paid', 'Completed')
df['status'] = df['status'].replace('cod', 'Completed')
df['status'] = df['status'].replace('order_refunded', 'Refund')
df['status'] = df['status'].replace('refund', 'Refund')
df['status'] = df['status'].replace('exchange', 'Refund')
df['status'] = df['status'].replace('pending', 'Pending')
df['status'] = df['status'].replace('payment_review', 'Pending')
df['status'] = df['status'].replace('processing', 'Pending')
df['status'] = df['status'].replace('holded', 'Pending')
df['status'] = df['status'].replace('pending_paypal', 'Pending')
df['status'] = df['status'].replace(r'\\N', 'Pending', regex=True)
df['status'] = df['status'].replace('fraud', 'Fraud')
df['status'] = df['status'].replace('canceled', 'Cancelled')

In [ ]:
df['status'].value_counts()

In [ ]:
df['BI Status'] = df['BI Status'].replace('#REF!', 'Net')

In [ ]:
df['BI Status'].value_counts()

##### Handling Null values in 'status' column

In [ ]:
df[df['status'].isnull()]

##### Observation
- 15 NaN values in 'status' column have 'Gross' in the BI column meaning all these transactions are not valid

##### Actions
- Replacing NaN values with label **'Cancelled'** in line with our understanding of the data

In [ ]:
df['status'].fillna("Cancelled",inplace=True)

#### Handling NaN values in 'category_name' column

In [ ]:
df['category_name'].value_counts()

##### Observations
- There are 164 NaN values in the **'category_name'** column that can be filled using some information from **'sku'** column. Not doing it right now
- 7850 transactions have a unicode label associated with them.
- 164 transactions have NaN values.

##### Actions
- Replacing the unicode label and NaN values with label 'Unknown'

In [ ]:
df['category_name'] = df['category_name'].replace(r'\\N', 'Unknown', regex=True)
df['category_name'].fillna("Unknown",inplace=True)

#### Handling NaN values in 'sku' column

In [ ]:
df[df['sku'].isnull()]

##### Obsevations
- 20 NaN values for **'sku'** exist in the dataset and these values can be replaced.

##### Action
- Replace NaN values with a new sku code **'Missing'**

In [ ]:
df['sku'].fillna("Missing",inplace=True)

#### Handling missing values in 'Sales_commission_code' column

In [ ]:
df['sales_commission_code'].value_counts()

In [ ]:
df[df['sales_commission_code'].isnull()]

##### Observations
- The column has a large number of NaN values and there are more than 7000 types of values in this column
- The column does not seem to add any value for further analysis and can be dropped at a later stage
- At this stage, NaN values as well as unicode labels can be replaced with 'Missing'

##### Actions
- Replacing NaN and unicode values with **'Missing'**

In [ ]:
df['sales_commission_code'].fillna("Missing",inplace=True)
df['sales_commission_code'] = df['sales_commission_code'].replace(r'\\N', 'Missing', regex=True)

#### Handling missing values in 'Customer ID' and 'Customer Since' columns

In [ ]:
df[df['Customer ID'].isnull()]

##### Observations
- There are a total of 11 rows where the 'Customer ID' column is NaN and exactly the same rows in 'Customer since' are also NaN, which makes sense and shows that these columns have a relationship.
- All 11 records are from FY18, with the first record from 01-2018.
- For keeping the records in dataset for analysis, a fake 'Customer ID' value of '0' can be assigned with '01-2018' assigned to all records in 'Customer Since' column

##### Actions
- Replaced 'Customer ID' with value **'0'** and 'Customer Since' with value **'01-2018'** for all NaN values

In [ ]:
df['Customer ID'].fillna("0",inplace=True)
df['Customer Since'].fillna("1-2018",inplace=True)

#### Checking for Null values again and setting appropriate datatypes

In [ ]:
df.isnull().sum()

#### Convert the datatypes of columns

In [ ]:
df[["item_id"]] = df[["item_id"]].astype("str")
df[["Month"]] = df[["Month"]].astype("int")
df[["Year"]] = df[["Year"]].astype("int")
df['created_at'] = pd.to_datetime(df['created_at'])
df[["qty_ordered"]] = df[["qty_ordered"]].astype("int")
df[["Customer ID"]] = df[["Customer ID"]].astype("int")
df[["increment_id"]] = df[["increment_id"]].astype("str")

## creating new columns to drill down the time dimension
df['day_of_week'] = df['created_at'].dt.dayofweek.astype(str) # 0 = monday.
#df['weekday_flag'] = (df['day_of_week'] // 5 != 1).astype(str)
df['date_of_month'] = df['created_at'].dt.day
df['Week'] = df['created_at'].dt.week

In [ ]:
df.info()

In [ ]:
df = df.reset_index()

## Step 2: Customer Segmentation

#### Combining some options for the payment method column to reduce the labels for this column

In [ ]:
df['payment_method'] = df['payment_method'].replace('Easypay_MA', 'Easypay')
df['payment_method'] = df['payment_method'].replace('cashatdoorstep', 'cod')
df['payment_method'] = df['payment_method'].replace(['marketingexpense','financesettlement','productcredit', 'internetbanking', 'mygateway', 'mcblite', 'ublcreditcard', 'apg'], 'Others')

#### The predictions would only be made for the 'completed' transactions

In [ ]:
df = df.loc[df['status']=='Completed',:].reset_index()
df.head()

#### Working only with completed transactions

In [ ]:
df_sales_segment = df.groupby('Customer ID')['grand_total'].sum().reset_index()
df_sales_segment

In [ ]:
df_purchases = df.groupby('Customer ID')['item_id'].count().reset_index()
df_purchases

In [ ]:
df_sales_segment.describe()

#### Total 80,122 unique customers did 'completed' transactions on the E-commerce store. 1 customer has spent Rs 35M with the next highest being close to Rs 5M. The customer who has spent Rs 35 has been considered as an outlier and not considered for further analysis.

In [ ]:
df_sales_segment.loc[df_sales_segment['grand_total'] > 5000000, :]
df_sales_segment = df_sales_segment.drop(37552, axis=0)
df_sales_segment

### Segmentation based on net amout spent by each customer on E-commerce store

A good estimate would be to use the values of 25th, 50th and 75th percentile. Based on these values, the 3 segments are
- very low: net amount spent less than Rs 1000
- low: net amount spent between Rs 1000 and Rs 10,000
- medium: net amount spent between Rs 10,000 and Rs 50,000
- high: net amount spent greater than Rs 50,000

In [ ]:
df_sales_segment['sales_segment'] = ''
df_sales_segment.loc[df_sales_segment['grand_total'] <= 1000, 'sales_segment'] = 'very low'
df_sales_segment.loc[(df_sales_segment['grand_total'] > 1000) & (df_sales_segment['grand_total'] <= 10000), 'sales_segment'] = 'low'
df_sales_segment.loc[(df_sales_segment['grand_total'] > 10000) & (df_sales_segment['grand_total'] <= 50000), 'sales_segment'] = 'medium'
df_sales_segment.loc[df_sales_segment['grand_total'] > 50000, 'sales_segment'] = 'high'
df_sales_segment

In [ ]:
sales_segment_total = df_sales_segment.groupby('sales_segment')['grand_total'].sum().reset_index()
sales_segment_total

In [ ]:
plt.figure(figsize=(15,6));
sns.countplot(x='sales_segment', order=['very low','low', 'medium','high'], data=df_sales_segment)

#### The biggest segment in terms of number of customers is the 'low' sales segment with almost 36,000 customers belonging to this segment. 'very low' and medium' segment have close to 20,000 customers and the 'high' sales segment the lowest with around 5000 customers

In [ ]:
plt.figure(figsize=(15,6));
sns.barplot(x='sales_segment', y='grand_total' ,order=['very low','low', 'medium', 'high'], data=sales_segment_total)

#### Highest sales revenue generated by the 'high' segment, followed by 'medium' sales segment. Least sales revenue is generated by the 'very low' segment, despite close to 20,000 customers belonging to this segment

#### Exploring each segment to check
- which items are purchased most by each segment?
- how many customers purchase more than one item during a single transaction?
- how many customers return to the store for doing more transactions?

### Very Low Sales segment

In [ ]:
df_vlow = df_sales_segment.loc[df_sales_segment['grand_total'] < 1900, ['Customer ID','grand_total']]
df_vlow

In [ ]:
df_vlow_segment = df_vlow.merge(df, how='inner', on='Customer ID')
df_vlow_segment = df_vlow_segment[['Customer ID','created_at','category_name','payment_method','grand_total_y']]
df_vlow_segment

In [ ]:
vlow_sales_segment_total = df_vlow_segment.groupby('category_name')['grand_total_y'].sum().reset_index()
vlow_sales_segment_total = vlow_sales_segment_total.sort_values(['grand_total_y'], ascending=False).reset_index(drop=True)
vlow_sales_segment_total

In [ ]:
plt.figure(figsize=(15,8));
sns.countplot(y='category_name', data=df_vlow_segment, order = df_vlow_segment['category_name'].value_counts().index)
plt.show()

In [ ]:
plt.figure(figsize=(15,8));
sns.barplot(y='category_name', data=vlow_sales_segment_total, x='grand_total_y', order = vlow_sales_segment_total['category_name'])
plt.show()

#### The product category with the highest sales is "Men's Fashion" followed by "Mobiles & Tablets" and "Beauty & Grooming"

In [ ]:
vlow_sales_segment_datewise = df_vlow_segment.groupby('created_at')['grand_total_y'].sum().reset_index()
vlow_sales_segment_datewise

In [ ]:
vlow_sales_segment_datewise.set_index('created_at')['grand_total_y'].plot(figsize=(15,8))

#### Biggest spike of Rs 0.5M seen on Black Friday sales in 2016. Smaller spikes seen around the annual deals offered on 'Black Friday' 2017 and around the holiday period of Eid-ul-Fitr

In [ ]:
vlow_sales_segment_payment = df_vlow_segment.groupby('payment_method')['grand_total_y'].sum().reset_index()
vlow_sales_segment_payment = vlow_sales_segment_payment.sort_values(['grand_total_y'], ascending=False).reset_index(drop=True)
vlow_sales_segment_payment

In [ ]:
plt.figure(figsize=(15,8));
sns.barplot(y='payment_method', data=vlow_sales_segment_payment, x='grand_total_y')
plt.show()

#### 'cod' or Cash on Delivery is the preferred payment method for this segment. Other payment methods have a very small contribution towards the total revenue.

In [ ]:
df_vlow_transactions = df_vlow_segment.groupby(['Customer ID','created_at']).size().reset_index(name='count')
df_vlow_transactions

In [ ]:
vlow_customer_count = df_vlow_transactions.loc[df_vlow_transactions['count'] > 1, :]
vlow_customer_count

In [ ]:
df_vlow_returning_customers = df_vlow_transactions.groupby('Customer ID')['created_at'].count().reset_index()
df_vlow_returning_customers

In [ ]:
vlow_customer_returning = df_vlow_returning_customers.loc[df_vlow_returning_customers['created_at'] > 1, :]
vlow_customer_returning

#### Out of 18,265 customers, only 1145 (6%) were returning customers and made more than one purchase on the E-commerce platform

### Low Sales segment

In [ ]:
df_low = df_sales_segment.loc[(df_sales_segment['grand_total'] > 1900) & (df_sales_segment['grand_total'] < 5750), ['Customer ID','grand_total']]
df_low

In [ ]:
df_low_segment = df_low.merge(df, how='inner', on='Customer ID')
df_low_segment = df_low_segment[['Customer ID','created_at','category_name','payment_method','grand_total_y']]
df_low_segment

In [ ]:
low_sales_segment_total = df_low_segment.groupby('category_name')['grand_total_y'].sum().reset_index()
low_sales_segment_total = low_sales_segment_total.sort_values(['grand_total_y'], ascending=False).reset_index(drop=True)
low_sales_segment_total

In [ ]:
plt.figure(figsize=(15,8));
sns.countplot(y='category_name', data=df_low_segment, order = df_low_segment['category_name'].value_counts().index)
plt.show()

In [ ]:
plt.figure(figsize=(15,8));
sns.barplot(y='category_name', data=low_sales_segment_total, x='grand_total_y', order = low_sales_segment_total['category_name'])
plt.show()

#### The product category with the highest revenue from sales is "Men's Fashion" followed by "Women's Fashion" and "Mobiles & Tablets"

In [ ]:
low_sales_segment_datewise = df_low_segment.groupby('created_at')['grand_total_y'].sum().reset_index()
low_sales_segment_datewise

In [ ]:
low_sales_segment_datewise.set_index('created_at')['grand_total_y'].plot(figsize=(15,8))

#### Big spikes of greater than Rs 3.0M spending seen on 'Black Friday' period in both 2016 and 2017. Also a spike of Rs 3.0M seen in the period of Feb-Mar 2018 that corresponds to some annual deals

In [ ]:
low_sales_segment_payment = df_low_segment.groupby('payment_method')['grand_total_y'].sum().reset_index()
low_sales_segment_payment = low_sales_segment_payment.sort_values(['grand_total_y'], ascending=False).reset_index(drop=True)
low_sales_segment_payment

In [ ]:
plt.figure(figsize=(15,8));
sns.barplot(y='payment_method', data=low_sales_segment_payment, x='grand_total_y')
plt.show()

#### 'cod' or Cash on Delivery is the most used method by customers belonging to this segment and almost 100M revenue out of total 136M is generated through this payment method.

In [ ]:
df_low_transactions = df_low_segment.groupby(['Customer ID','created_at']).size().reset_index(name='count')
df_low_transactions

In [ ]:
low_customer_count = df_low_transactions.loc[df_low_transactions['count'] > 1, :]
low_customer_count

In [ ]:
df_low_returning_customers = df_low_transactions.groupby('Customer ID')['created_at'].count().reset_index()
df_low_returning_customers

In [ ]:
low_customer_returning = df_low_returning_customers.loc[df_low_returning_customers['created_at'] > 1, :]
low_customer_returning

#### Out of 37,105 customers in this segment, 9,167 (25%) were returning customers and made more than one purchase on the E-commerce store

### Medium Sales segment

In [ ]:
df_medium = df_sales_segment.loc[(df_sales_segment['grand_total'] > 5750) & (df_sales_segment['grand_total'] < 23000), ['Customer ID','grand_total']]
df_medium

In [ ]:
df_medium_segment = df_medium.merge(df, how='inner', on='Customer ID')
df_medium_segment = df_medium_segment[['Customer ID','created_at','category_name','payment_method','grand_total_y']]
df_medium_segment

In [ ]:
plt.figure(figsize=(15,8));
sns.countplot(y='category_name', data=df_medium_segment, order = df_medium_segment['category_name'].value_counts().index)
plt.show()

In [ ]:
medium_sales_segment_total = df_medium_segment.groupby('category_name')['grand_total_y'].sum().reset_index()
medium_sales_segment_total = medium_sales_segment_total.sort_values(['grand_total_y'], ascending=False).reset_index(drop=True)
medium_sales_segment_total

In [ ]:
plt.figure(figsize=(15,8));
sns.barplot(y='category_name', data=medium_sales_segment_total, x='grand_total_y', order = medium_sales_segment_total['category_name'])
plt.show()

#### Mobiles & Tablets generate the highest amount of revenue for the 'medium' sales segment of customers, which is about 3 times higher than the 2nd best category 'Appliances'

In [ ]:
medium_sales_segment_datewise = df_medium_segment.groupby('created_at')['grand_total_y'].sum().reset_index()
medium_sales_segment_datewise

In [ ]:
medium_sales_segment_datewise.set_index('created_at')['grand_total_y'].plot(figsize=(15,8))

#### Customers belonging to this segment were very active on annual Black Friday in November 2017. Another active period was Mar-Apr 2018 and significant activity on Black Friday 2016 and Eid-ul-Fitr 2017.

In [ ]:
medium_sales_segment_payment = df_medium_segment.groupby('payment_method')['grand_total_y'].sum().reset_index()
medium_sales_segment_payment = medium_sales_segment_payment.sort_values(['grand_total_y'], ascending=False).reset_index(drop=True)
medium_sales_segment_payment

In [ ]:
plt.figure(figsize=(15,8));
sns.barplot(y='payment_method', data=medium_sales_segment_payment, x='grand_total_y')
plt.show()

#### Out of 394M, almost 175M is collected through 'cod' or Cash on Delivery payment method but other payment methods have a better share for this segment than 'very low' and 'low' sales segments

In [ ]:
df_medium_transactions = df_medium_segment.groupby(['Customer ID','created_at']).size().reset_index(name='count')
df_medium_transactions

In [ ]:
medium_customer_count = df_medium_transactions.loc[df_medium_transactions['count'] > 1, :]
medium_customer_count

In [ ]:
df_medium_returning_customers = df_medium_transactions.groupby('Customer ID')['created_at'].count().reset_index()
df_medium_returning_customers

In [ ]:
medium_customer_returning = df_medium_returning_customers.loc[df_medium_returning_customers['created_at'] > 1, :]
medium_customer_returning

#### In the 'medium' sales segment, out of 83,389 transactions made by 17,385 customers, 7,278 or 42% of the customers made more than one purchase or were returning customers

### High Sales segment

In [ ]:
df_high = df_sales_segment.loc[df_sales_segment['grand_total'] > 23000, ['Customer ID','grand_total']]
df_high

In [ ]:
df_high_segment = df_high.merge(df, how='inner', on='Customer ID')
df_high_segment = df_high_segment[['Customer ID','created_at','category_name','payment_method','grand_total_y']]
df_high_segment

In [ ]:
plt.figure(figsize=(15,8));
sns.countplot(y='category_name', data=df_high_segment, order = df_high_segment['category_name'].value_counts().index)
plt.show()

#### Mobiles & Tablets is the category which has highest number of transactions for 'high' sales segment

In [ ]:
high_sales_segment_total = df_high_segment.groupby('category_name')['grand_total_y'].sum().reset_index()
high_sales_segment_total = high_sales_segment_total.sort_values(['grand_total_y'], ascending=False).reset_index(drop=True)
high_sales_segment_total

In [ ]:
plt.figure(figsize=(15,8));
sns.barplot(y='category_name', data=high_sales_segment_total, x='grand_total_y', order = high_sales_segment_total['category_name'])
plt.show()

#### 'Mobiles & Tablets' is the most favourite product category in terms of revenue generated by the 'high' sales segment group and almost 3 times more revenue is generated as compared to the next product category 'Appliances. The pattern is very much similar to the pattern seen in the 'medium' sales segment

In [ ]:
high_sales_segment_datewise = df_high_segment.groupby('created_at')['grand_total_y'].sum().reset_index()
high_sales_segment_datewise

In [ ]:
high_sales_segment_datewise.set_index('created_at')['grand_total_y'].plot(figsize=(15,8))

#### Big spike of more than Rs 30M seen on annual Black Friday sales in 2017 with smaller spikes of greater than Rs 10M seen on other public holidays and annual sales

In [ ]:
high_sales_segment_payment = df_high_segment.groupby('payment_method')['grand_total_y'].sum().reset_index()
high_sales_segment_payment = high_sales_segment_payment.sort_values(['grand_total_y'], ascending=False).reset_index(drop=True)
high_sales_segment_payment

In [ ]:
plt.figure(figsize=(15,8));
sns.barplot(y='payment_method', data=high_sales_segment_payment, x='grand_total_y')
plt.show()

#### 'cod' or Cash on Delivery still contributes about 1/4th of the total revenue generated by this segment but other digital payment methods like Easypay_voucher, Payaxis, Easypay and bankalfalah have significant share in the overall revenue.

In [ ]:
df_high_transactions = df_high_segment.groupby(['Customer ID','created_at']).size().reset_index(name='count')
df_high_transactions

In [ ]:
high_customer_count = df_high_transactions.loc[df_high_transactions['count'] > 1, :]
high_customer_count

In [ ]:
df_high_returning_customers = df_high_transactions.groupby('Customer ID')['created_at'].count().reset_index()
df_high_returning_customers

In [ ]:
high_customer_returning = df_high_returning_customers.loc[df_high_returning_customers['created_at'] > 1, :]
high_customer_returning

#### In the 'high' sales segment, out of 125,096 transactions made by 6,431 customers, 4,368 or 68% of the customers made more than one purchase or were returning customers

## Summary

#### Very Low Sales segment
- Total customers: 18,265
- Total Revenue generated: Rs.10.7 Million
- 6% were returning customers on the E-commerce platform
- Average revenue generated per customer: Rs 590
- Top 3 revenue generating product categories are
    - Men's Fashion
    - Mobiles & Tablets
    - Beauty & Grooming

#### Low Sales segment
- Total customers: 37,105
- Total Revenue generated: Rs.136 Million
- 25% were returning customers on the E-commerce platform
- Average revenue generated per customer: Rs 3,747
- Top 3 revenue generating product categories are
    - Men's Fashion
    - Women's Fashion
    - Mobiles & Tablets

#### Medium  Sales segment
- Total customers: 17,385
- Total Revenue generated: Rs.394 Million
- 42% were returning customers on the E-commerce platform
- Average revenue generated per customer: Rs 22,713
- Top 3 revenue generating product categories are
    - Mobiles & Tablets
    - Appliances
    - Entertainment


#### High  Sales segment
- Total customers: 6,431
- Total Revenue generated: Rs.1,054 Million
- 68% were returning customers on the E-commerce platform
- Average revenue generated per customer: Rs 163,997.
- Top 3 revenue generating product categories are
    - Mobiles & Tablets
    - Appliances
    - Entertainment

## Segmentation based on sales in different categories

In [ ]:
pivoted= pd.pivot_table(df,index='Customer ID',columns='category_name', values='grand_total',aggfunc='sum',fill_value=0)

In [ ]:
pivoted

In [ ]:
pivoted.dtypes

In [ ]:
new = pivoted.reset_index()
new

In [ ]:
X= new.drop('Customer ID',axis=1)
X.head()

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics

In [ ]:
# generating random clutser with value set to 5
kmeans = KMeans(n_clusters=5, random_state=0)
kmeans.fit(X)

In [ ]:
pred = X.copy()
pred['kmean1'] = kmeans.labels_
pred.head()

In [ ]:
col_names = X.columns
col_names

##### Performing scaling for implementation of Kmeans

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaled = scaler.fit_transform(X)

In [ ]:
scaled=pd.DataFrame(columns=col_names,data=scaled)

In [ ]:
scaled.head()

#### Choosing K for Kmeans

In [ ]:
cost = []
# run for cluster 1 to 15
for k in range(1, 15):
    kmeanModel = KMeans(n_clusters=k, random_state=0).fit(scaled)
    cost.append([k,kmeanModel.inertia_])

In [ ]:
plt.figure(figsize=(15,6))
sns.set_context('poster')
plt.plot(pd.DataFrame(cost)[0], pd.DataFrame(cost)[1])
plt.xlabel('k')
plt.ylabel('Cost')
plt.title('The Elbow Method showing the optimal k')
plt.show()

##### The plot shows that optimal method for K is 4

In [ ]:
#pred = scaled.copy()

#Write code here
kmean4 = KMeans(n_clusters=4, random_state=0)

#write code to fit
kmean4.fit(scaled)

#Write code to assign labels to predicted data
pred['kmean4'] = kmean4.labels_

#View the final data set i.e top 5 rows
pred.head()

In [ ]:
pred = pred.drop('kmean1',axis=1)
pred.head()

In [ ]:
pivoted = pred.groupby('kmean4')["Appliances", "Beauty & Grooming","Books","Computing","Entertainment", "Health & Sports", "Home & Living","Kids & Baby","Men's Fashion","Mobiles & Tablets","Others","School & Education","Soghaat","Superstore","Unknown","Women's Fashion"].sum().reset_index()
pivoted

In [ ]:
pivoted['Name'] = ''
pivoted.loc[pivoted['kmean4'] == 0, 'Name'] = 'Cluster 0'
pivoted.loc[pivoted['kmean4'] == 1, 'Name'] = 'Cluster 1'
pivoted.loc[pivoted['kmean4'] == 2, 'Name'] = 'Cluster 2'
pivoted.loc[pivoted['kmean4'] == 3, 'Name'] = 'Cluster 3'

In [ ]:
import re
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

In [ ]:
plt_cols = ["Appliances", "Beauty & Grooming","Books","Computing","Entertainment", "Health & Sports", "Home & Living","Kids & Baby","Men's Fashion","Mobiles & Tablets","Others","School & Education","Soghaat","Superstore","Unknown","Women's Fashion","Appliances"]
#top_fwds.reset_index(inplace=True)

plt_data = []
for i in range(0,1):
    trace = go.Scatterpolar(
        r = pivoted.loc[i,plt_cols],
        theta = plt_cols,
        #mode = 'lines',
        name = pivoted.loc[i,'Name'],
    )
    plt_data.append(trace)

layout = go.Layout(
  polar = dict(
    radialaxis = dict(
      visible = True,
      range = [0, 550000000],
    )
  ),
    height = 900,
    width = 900,
    title = "Segments based on Category Name",
    showlegend = True
)

fig = go.Figure(data=plt_data, layout=layout)
iplot(fig)

In [ ]:
plt_cols = ["Appliances", "Beauty & Grooming","Books","Computing","Entertainment", "Health & Sports", "Home & Living","Kids & Baby","Men's Fashion","Mobiles & Tablets","Others","School & Education","Soghaat","Superstore","Unknown","Women's Fashion","Appliances"]
#top_fwds.reset_index(inplace=True)

plt_data = []
for i in range(1,2):
    trace = go.Scatterpolar(
        r = pivoted.loc[i,plt_cols],
        theta = plt_cols,
        #mode = 'lines',
        name = pivoted.loc[i,'Name'],
    )
    plt_data.append(trace)

layout = go.Layout(
  polar = dict(
    radialaxis = dict(
      visible = True,
      range = [0, 20000000],
    )
  ),
    height = 900,
    width = 900,
    title = "Segments based on Category Name",
    showlegend = True
)

fig = go.Figure(data=plt_data, layout=layout)
iplot(fig)

In [ ]:
plt_cols = ["Appliances", "Beauty & Grooming","Books","Computing","Entertainment", "Health & Sports", "Home & Living","Kids & Baby","Men's Fashion","Mobiles & Tablets","Others","School & Education","Soghaat","Superstore","Unknown","Women's Fashion","Appliances"]
#top_fwds.reset_index(inplace=True)

plt_data = []
for i in range(2,3):
    trace = go.Scatterpolar(
        r = pivoted.loc[i,plt_cols],
        theta = plt_cols,
        #mode = 'lines',
        name = pivoted.loc[i,'Name'],
    )
    plt_data.append(trace)

layout = go.Layout(
  polar = dict(
    radialaxis = dict(
      visible = True,
      range = [0, 80000000],
    )
  ),
    height = 900,
    width = 900,
    title = "Segments based on Category Name",
    showlegend = True
)

fig = go.Figure(data=plt_data, layout=layout)
iplot(fig)

In [ ]:
plt_cols = ["Appliances", "Beauty & Grooming","Books","Computing","Entertainment", "Health & Sports", "Home & Living","Kids & Baby","Men's Fashion","Mobiles & Tablets","Others","School & Education","Soghaat","Superstore","Unknown","Women's Fashion","Appliances"]
#top_fwds.reset_index(inplace=True)

plt_data = []
for i in range(3,4):
    trace = go.Scatterpolar(
        r = pivoted.loc[i,plt_cols],
        theta = plt_cols,
        #mode = 'lines',
        name = pivoted.loc[i,'Name'],
    )
    plt_data.append(trace)

layout = go.Layout(
  polar = dict(
    radialaxis = dict(
      visible = True,
      range = [0, 25000000],
    )
  ),
    height = 900,
    width = 900,
    title = "Segments based on Category Name",
    showlegend = True
)

fig = go.Figure(data=plt_data, layout=layout)
iplot(fig)

## Summary
The clustering algorithm focuses on segmentation based on spending within a particular product category and hence the results are a bit different from the overall spending. However, it gives a different insight into the spending patterns of customer segement witin the dataset.